In [29]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta
from openpyxl import load_workbook

In [30]:
%run "./technical_analysis_self.ipynb"

In [31]:
def get_returns(df):
    for i in [2, 3, 5, 8, 13]:
        df[f'd{i}_high'] = df['high'].shift(-i)
        df[f'd{i}_close'] = df['close'].shift(-i)
        df[f'd{i}_return_pct'] = (df[f'd{i}_close'] - df['next_day_open']) * 100 / df['next_day_open']
        df[f'd{i}_win_flag'] = df[f'd{i}_return_pct'] >= 2
    return df

def dump_to_excel(df, path, indicator_column):
    start_row = 0
    col_set = [indicator_column] + ['symbol', 'next_day_open', 'ADX', 'plus_DI', 'minus_DI'] 
    col_set += [i for i in df.columns if '_return_pct' in i or '_flag' in i]
    
    write_df = df[df[indicator_column].notna()][col_set]
    
    try:
        wb = load_workbook(path)
        start_row = len(wb[indicator_column]['A'])
    except:
        pass
    
    try:
        with pd.ExcelWriter(path, mode='a', if_sheet_exists='overlay') as excel_writer:
            write_df.to_excel(excel_writer, sheet_name=indicator_column, startrow=start_row, header=start_row == 0) 
    except FileNotFoundError:
        write_df.to_excel(path, sheet_name=indicator_column, startrow=start_row) 

In [32]:
path_to_excel = './backtest_2.xlsx'

In [21]:
excel_df = None

for script in ['infy', 'wipro', 'tcs', 'canbk', 'tatasteel', 'ashokley', 'oil', 'tejasnet', 'bhartiartl', 
               'hindpetro', 'railtel', 'zomato', 'hfcl', 'rain', 'paytm', 'biocon', 'bankbaroda', 'tatapower',
               'ambujacem', 'hcltech', 'sbin', 'tatamotors', 'coalindia', 'jswsteel', 'itc', 'hdfc'
              ]:
    df = YFinanceDataLoader.load_data(script, 100)
    df["next_day_open"] = df["open"].shift(-1)
    df = load_ADX(load_ATR(df), 14)
    for fn in [preprocess, get_returns, identify_marubozus, identify_engulfing, identify_haramis, 
              identify_paper_umbrellas, identify_piercing_or_dark_clouds, identify_shooting_stars]:
        df = fn(df)
        
    excel_df = df if excel_df is None else pd.concat([excel_df, df])

In [23]:
for indicator_column in ['marubozu', 'engulfing', 'harami', 'partial_engulfing', 'paper_umbrella_type']:
    dump_to_excel(excel_df, path_to_excel, indicator_column)

In [28]:
excel_df.groupby(['marubozu', 'symbol']).agg({
    'marubozu': 'count',
    'd2_return_pct': 'mean',
    'd3_return_pct': 'mean',
    'd5_return_pct': 'mean',
    'd8_return_pct': 'mean',
    'd13_return_pct': 'mean'
})

marubozu  d2_return_pct  d3_return_pct  d5_return_pct  \
marubozu symbol                                                              
Bearish  ASHOKLEY           3       2.203641       1.386295       2.040430   
         BANKBARODA         1       1.676188       2.247617       5.180955   
         BHARTIARTL         3      -1.079759       0.214857       2.569010   
         BIOCON             3       0.120431      -0.711177       1.551174   
         CANBK              3       0.331110       2.593826       4.156578   
         COALINDIA          2      -1.914933       1.818635       1.551644   
         HCLTECH            4      -1.162284       0.039610      -0.085267   
         HDFC               1       1.756319       1.214498       2.458838   
         HFCL               4       1.962739       0.508174       0.115895   
         HINDPETRO          3      -0.473232      -1.048522      -4.163927   
         INFY               3      -0.880074       0.498463      -0.990993   
         ITC                2      -0.290692       0.553186      -3.164181   
         JSWSTEEL           2       0.339877       2.630824       2.206542   
         OIL                3      -2.755920      -2.540586      -1.151602   
         PAYTM              1       3.505054       5.849602       9.150784   
         RAILTEL            2      -0.052355       1.503123       2.867991   
         RAIN               3      -1.319295       1.071180      -0.584361   
         SBIN               1       2.972948       2.638907       2.702765   
         TATAMOTORS         2       0.252323       2.149302       2.498516   
         TATAPOWER          3      -1.881277      -0.329133      -2.401184   
         TATASTEEL          6       0.364509       1.249868      -1.491434   
         TCS                5      -0.497911       2.507324       3.250645   
         TEJASNET           3      -0.449255      -0.845777      -1.577636   
         WIPRO             10       0.566628       1.207245       1.136233   
         ZOMATO             5      -1.280579      -0.984533       4.592963   
Bullish  ASHOKLEY           4       1.582371       2.778225       2.534103   
         BANKBARODA         5       0.065438       1.101553       2.707379   
         BHARTIARTL         1       0.455209      -4.544801      -7.008664   
         BIOCON             2      -4.185829      -5.218533      -2.645815   
         CANBK              3       1.967380       1.394378       0.488595   
         COALINDIA          2      -4.113674      -4.438429       0.893093   
         HCLTECH            1       1.335613       1.736297       2.275883   
         HDFC               5       0.004480       0.109622       0.297217   
         HINDPETRO          1       2.684257       0.022750       1.160150   
         INFY               2       0.422455       1.995175       1.729420   
         ITC                1       1.142408       2.926449       2.065730   
         JSWSTEEL           5       0.767672       1.495021       1.818612   
         PAYTM              1       0.498469      -3.767003      -8.174895   
         RAIN               2       1.624364       0.913707       1.522843   
         SBIN               4       1.563107       2.272314       2.442805   
         TATAMOTORS         1      -2.317813      -2.088119      -2.223845   
         TATAPOWER          1      -0.830129      -1.862722      -2.389145   
         TATASTEEL          1      -0.176292       0.010687       1.447722   
         TCS                2       0.971626       0.975094       0.843418   
         ZOMATO             1      -4.883234      -5.095539      -6.510967   

                     d8_return_pct  d13_return_pct  
marubozu symbol                                     
Bearish  ASHOKLEY         5.724038        5.976672  
         BANKBARODA       7.009521        7.695243  
         BHARTIARTL       3.541767        5.825113  
         BIOCON           1.581230             NaN  
         CANBK            6.557848  

In [33]:
class DataLoader:
    req_columns = ['CH_TIMESTAMP', 'CH_SYMBOL', 'CH_TRADE_HIGH_PRICE', 'CH_TRADE_LOW_PRICE', 'CH_OPENING_PRICE', 'CH_CLOSING_PRICE', 'CH_LAST_TRADED_PRICE', 'CH_PREVIOUS_CLS_PRICE', 'CH_TOT_TRADED_QTY', 'CH_52WEEK_HIGH_PRICE', 'CH_52WEEK_LOW_PRICE']
    new_column_names = ['date', 'symbol', 'high', 'low', 'open', 'close', 'ltp', 'prev_close', 'volume', 'high_52w', 'low_52w']

    @staticmethod
    def load_data(script_name, start_date, end_date, series="EQ"):
        df = equity_history(script_name, series, start_date, end_date)[DataLoader.req_columns]
        df.columns = DataLoader.new_column_names
        return df

In [34]:
script_name = 'infy'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 40)

In [35]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:561: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,end_date))
